# Inspecting Ecore file for SysML v2 Metamodel

This notebook steps through a couple of different methods for loading up the SysML v2 Ecore metamodel. One with a library called pyEcore and another just using the raw lxml library.

In [ ]:
import json

In [ ]:
from pyecore.resources import ResourceSet, URI
from pyecore.ecore import EClass, EAttribute, EString, EObject, EProxy, EEnum

In [ ]:
# From the quick start for PyEcore to load a file into the metaclass model

rset = ResourceSet()
sysml_resource = rset.get_resource(URI('metamodel/SysML.ecore'))
mm_root = sysml_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root

types_resource = rset.get_resource(URI('metamodel/types.ecore'))
mm_root2 = types_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root2

In [ ]:
mm_root

Determine what the fields for an EPackage in this framework are:

In [ ]:
dir(mm_root)

In [ ]:
all_classes = mm_root.eClassifiers

Determine what the fields for an EClass in this framework are:

In [ ]:
dir(all_classes[0])

In [ ]:
def get_eclass(eclass_name):
    return [the_class for the_class in all_classes if the_class.name == eclass_name][0]

In [ ]:
owning_membership = [the_class for the_class in all_classes if the_class.name == "OwningMembership"]

In [ ]:
owning_membership[0]

In [ ]:
owning_membership[0].eAttributes

In [ ]:
owning_membership[0].eReferences

In [ ]:
owning_membership[0].eAttributes[0]

In [ ]:
dir(owning_membership[0].eAttributes[0])

In [ ]:
owning_membership[0].eAttributes[0].derived

In [ ]:
attr_type = None
if isinstance(owning_membership[0].eAttributes[0].eType, EProxy):
    attr_type = owning_membership[0].eAttributes[0].eType.force_resolve()
else:
    attr_type = owning_membership[0].eAttributes[0].eType

In [ ]:
attr_type.name

In [ ]:
[super_type.name for super_type in all_classes[0].eAllSuperTypes()]

In [ ]:
owning_membership[0].eAllAttributes()

In [ ]:
owning_membership[0].eAllReferences()

In [ ]:
list(owning_membership[0].eAllReferences())[10].lowerBound

In [ ]:
def element_has_feature_by_name(element, name):
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    for ref in all_refs:
        if ref.name == name:
            return True
        
    for att in all_atts:
        if att.name == name:
            return True
        
    return False

In [ ]:
element_has_feature_by_name(owning_membership[0], "relatedElement")

In [ ]:
def layout_features(element):
    
    feature_line = []
    
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    local_refs = element.eReferences
    local_atts = element.eAttributes
    
    for ref in all_refs:
        new_line = []
        new_line.append(ref.name)
        if ref in local_refs:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if ref.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(ref.eType, EProxy):
            new_line.append(ref.eType.force_resolve().name)
        else:
            new_line.append(ref.eType.name)
        
        new_line.append('EReference')
        
        lb = ref.lowerBound
        ub = ref.upperBound
        
        new_line.append(lb)
        new_line.append(ub)
        
        feature_line.append(new_line)
        
    for att in all_atts:
        new_line = []
        new_line.append(att.name)
        if att in local_atts:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if att.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(att.eType, EProxy):
            new_line.append(att.eType.force_resolve().name)
        else:
            new_line.append(att.eType.name)
        
        new_line.append('EAttribute')
        
        lb = ref.lowerBound
        ub = ref.upperBound
        
        new_line.append(lb)
        new_line.append(ub)
        
        feature_line.append(new_line)
    
    return feature_line

In [ ]:
layout_features(owning_membership[0])

In [ ]:
def get_primary_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = [feature for feature in all_features if feature[2] == 'primary']
    
    return primary_features

In [ ]:
def get_derived_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = [feature for feature in all_features if feature[2] == 'derived']
    
    return primary_features

In [ ]:
layout_features(get_eclass("OwningMembership"))

In [ ]:
layout_features(get_eclass("PartUsage"))

In [ ]:
def find_all_sub_types(element):
    # go through all of the meta-elements in the Ecore and find the ones that have the 
    # element given as an argument as one of their super types
    
    sub_types = []
    
    all_classes = mm_root.eClassifiers
    for clz in all_classes:
        if not isinstance(clz, EEnum):
            if element in clz.eAllSuperTypes():
                sub_types.append(clz)
            
    return sub_types

In [ ]:
find_all_sub_types(get_eclass("OwningMembership"))

In [ ]:
get_primary_attributes("PartDefinition")

In [ ]:
get_derived_attributes("PartUsage")

In [ ]:
get_primary_attributes("OwningMembership")

In [ ]:
get_primary_attributes("Membership")

In [ ]:
get_primary_attributes("Specialization")

In [ ]:
get_primary_attributes("FeatureTyping")

In [ ]:
attributes_dict = {k.name:get_primary_attributes(k.name) for k in all_classes if isinstance(k, EClass)}

In [ ]:
file1 = open("metamodel/sysml_ecore_atts.json","w")

In [ ]:
json.dump(attributes_dict, file1)

In [ ]:
file1.close()

In [ ]:
refs_dict = {k.name:get_derived_attributes(k.name) for k in all_classes if isinstance(k, EClass)}

In [ ]:
file2 = open("metamodel/sysml_ecore_derived_refs.json","w")

In [ ]:
json.dump(refs_dict, file2)

In [ ]:
file2.close()